# Query and Join Data
This notebook demonstrates how to query and join data from the models to resemble an imported CSV file. Missing data will be represented as NaN.

In [19]:
from models import Airline, Airport, Airplane, Flight, FlightDetails
from sqlalchemy.orm import sessionmaker, aliased
from sqlalchemy import create_engine
import pandas as pd

# Set up the database connection
engine = create_engine("sqlite:///flights.db")
Session = sessionmaker(bind=engine)
session = Session()

In [20]:
# Alias the Airport table for departure and arrival airports
DepartureAirport = aliased(Airport)
ArrivalAirport = aliased(Airport)

# Query and join data using SQLAlchemy ORM with outer joins
query = (
    session.query(
        Flight.id.label("flight_id"),
        Flight.status,
        FlightDetails.flight_number,
        FlightDetails.call_sign,
        Airline.airline_name,
        Airline.airline_iata,
        Airline.airline_icao,
        Airplane.aircraft_model,
        Airplane.aircraft_reg,
        DepartureAirport.airport_name.label("departure_airport_name"),
        DepartureAirport.airport_iata.label("departure_airport_iata"),
        DepartureAirport.airport_icao.label("departure_airport_icao"),
        DepartureAirport.timezone.label("departure_timezone"),
        ArrivalAirport.airport_name.label("arrival_airport_name"),
        ArrivalAirport.airport_iata.label("arrival_airport_iata"),
        ArrivalAirport.airport_icao.label("arrival_airport_icao"),
        ArrivalAirport.timezone.label("arrival_timezone"),
        Flight.dep_date_time_UTC.label("departure_datetime"),
        Flight.dep_rev_date_time_UTC.label("departure_datetime_revised"),
        Flight.arr_date_time_UTC.label("arrival_datetime"),
        Flight.arr_rev_date_time_UTC.label("arrival_datetime_revised"),
    )
    .outerjoin(FlightDetails, Flight.flight_details_id == FlightDetails.id)
    .outerjoin(Airline, Flight.arline_id == Airline.id)  # Corrected typo
    .outerjoin(Airplane, Flight.airplane_id == Airplane.id)
    .outerjoin(DepartureAirport, Flight.dep_airport_id == DepartureAirport.id)
    .outerjoin(ArrivalAirport, Flight.arr_airport_id == ArrivalAirport.id)
)

# Convert the query result to a Pandas DataFrame
joined_data = pd.DataFrame(query.all(), columns=[
    "flight_id", "status", "flight_number", "call_sign", "airline_name", 
    "airline_iata", "airline_icao", "aircraft_model", "aircraft_reg", 
    "departure_airport_name", "departure_airport_iata", "departure_airport_icao", 
    "departure_timezone", "arrival_airport_name", "arrival_airport_iata", 
    "arrival_airport_icao", "arrival_timezone", "departure_datetime", "departure_datetime_revised", 
    "arrival_datetime", "arrival_datetime_revised"
])

In [21]:
joined_data.head()

,flight_id,status,flight_number,call_sign,airline_name,airline_iata,airline_icao,aircraft_model,aircraft_reg,departure_airport_name,...,departure_airport_icao,departure_timezone,arrival_airport_name,arrival_airport_iata,arrival_airport_icao,arrival_timezone,departure_datetime,departure_datetime_revised,arrival_datetime,arrival_datetime_revised
0,1,Arrived,FR 2228,RYR8447,Ryanair,FR,RYR,Boeing 737,SP-RZE,Valencia,...,LEVC,Europe/Madrid,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 18:20:00,2025-04-29 18:24:00,2025-04-29 21:20:00,2025-04-30 21:23:00
1,2,Arrived,W6 2068,WZZ976,Wizz Air,W6,WZZ,Airbus A321 NEO,9H-WNJ,Milan,...,LIMC,Europe/Rome,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 20:00:00,2025-04-29 20:18:00,2025-04-29 21:55:00,2025-04-30 21:48:00
2,3,Arrived,W6 2092,WZZ677,Wizz Air,W6,WZZ,Airbus A320,HA-LXV,Lyon,...,LFLL,Europe/Paris,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 19:45:00,2025-04-29 19:55:00,2025-04-29 21:55:00,2025-04-30 21:55:00
3,4,Arrived,FR 6783,RYR4LX,Ryanair,FR,RYR,Boeing 737-800,SP-RKO,Trieste,...,LIPQ,Europe/Rome,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 21:00:00,2025-04-29 21:00:00,2025-04-29 22:30:00,2025-04-30 22:08:00
4,5,Arrived,FR 2778,None,Ryanair,FR,RYR,Boeing 737-800,None,Thessaloniki,...,LGTS,Europe/Athens,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 20:55:00,NaT,2025-04-29 22:55:00,2025-04-30 22:31:00


In [22]:
# Select specific columns to keep
columns_to_keep = [
    "flight_number",
    "call_sign",
    "status",
    "airline_name",
    "airline_iata",
    "airline_icao",
    "aircraft_reg",
    "aircraft_model",
    "departure_airport_name",
    "departure_airport_iata",
    "departure_airport_icao",
    "departure_timezone",
    "departure_datetime",
    "departure_datetime_revised",
    "arrival_airport_name",
    "arrival_airport_iata",
    "arrival_airport_icao",
    "arrival_timezone",
    "arrival_datetime",
    "arrival_datetime_revised",
]

cleaned_data = joined_data[columns_to_keep]

In [23]:
cleaned_data.head()

,flight_number,call_sign,status,airline_name,airline_iata,airline_icao,aircraft_reg,aircraft_model,departure_airport_name,departure_airport_iata,departure_airport_icao,departure_timezone,departure_datetime,departure_datetime_revised,arrival_airport_name,arrival_airport_iata,arrival_airport_icao,arrival_timezone,arrival_datetime,arrival_datetime_revised
0,FR 2228,RYR8447,Arrived,Ryanair,FR,RYR,SP-RZE,Boeing 737,Valencia,VLC,LEVC,Europe/Madrid,2025-04-29 18:20:00,2025-04-29 18:24:00,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 21:20:00,2025-04-30 21:23:00
1,W6 2068,WZZ976,Arrived,Wizz Air,W6,WZZ,9H-WNJ,Airbus A321 NEO,Milan,MXP,LIMC,Europe/Rome,2025-04-29 20:00:00,2025-04-29 20:18:00,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 21:55:00,2025-04-30 21:48:00
2,W6 2092,WZZ677,Arrived,Wizz Air,W6,WZZ,HA-LXV,Airbus A320,Lyon,LYS,LFLL,Europe/Paris,2025-04-29 19:45:00,2025-04-29 19:55:00,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 21:55:00,2025-04-30 21:55:00
3,FR 6783,RYR4LX,Arrived,Ryanair,FR,RYR,SP-RKO,Boeing 737-800,Trieste,TRS,LIPQ,Europe/Rome,2025-04-29 21:00:00,2025-04-29 21:00:00,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 22:30:00,2025-04-30 22:08:00
4,FR 2778,None,Arrived,Ryanair,FR,RYR,None,Boeing 737-800,Thessaloniki,SKG,LGTS,Europe/Athens,2025-04-29 20:55:00,NaT,Kraków,KRK,EPKK,Europe/Warsaw,2025-04-29 22:55:00,2025-04-30 22:31:00


In [24]:
cleaned_data.isnull().sum()

flight_number                   0
call_sign                     118
status                          0
airline_name                    0
airline_iata                   22
airline_icao                   15
aircraft_reg                  123
aircraft_model                 12
departure_airport_name          5
departure_airport_iata          5
departure_airport_icao          5
departure_timezone              5
departure_datetime             17
departure_datetime_revised    141
arrival_airport_name           20
arrival_airport_iata           20
arrival_airport_icao           20
arrival_timezone               20
arrival_datetime              100
arrival_datetime_revised      275
dtype: int64

In [25]:
cleaned_data.to_csv("csv_retrieved/cleaned_flight_data.csv", index=False)